# Demo and Comparission of Big Data File Formats
This Notebook runs only on a local Spark Environment, not on Kubernetes

## 1. CSV and JSON
Old dat formats that are not designed for big data and scaling  
**Typical feature:** humand readable

## 2. Avro, OCR, Parquet
First generation of special big data formats that allow fast writes, fast reads or both  
**Typical features:** splittable, compressible, data skipping and predicat pushdown, data schema inclueded



## 3. Delta, Iceberg, Hudi
Latest generation of big data format that support ACID transaction, audit save transaction logs and time travel  
**Typical features:** enhancing first generation format with additonal meta data and read/write procedures.

In [1]:
#################################################################################
# Laden aller relevate Module
#################################################################################
import pyspark
from pyspark.sql.functions import *
import json
import csv
from datetime import datetime
from delta import *
import delta

# use 95% of the screen for jupyter cell
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }<style>"))

In [2]:
# first for local usage pip install delta-spark

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars", "/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar") \
    .config("spark.driver.extraClassPath","/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar") \
    .config("spark.executor.extraClassPath","/Users/alor/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.3.1.jar, /Users/alor/opt/spark/jars/kafka-clients-3.3.1.jar, /Users/alor/opt/spark/jars/spark-avro_2.12-3.3.1.jar")


spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [3]:
spark

## Create sample data

In [4]:
# initial daten
account_data1 = [
    (1,"alex","2019-01-01",1000),
    (2,"alex","2019-02-01",1500),
    (3,"alex","2019-03-01",1700),
    (4,"maria","2020-01-01",5000)
    ]

# update mit Änderung und neuem Datensat
account_data2 = [
    (1,"alex","2019-03-01",3300),
    (2,"peter","2021-01-01",100)
    ]

# Update mit neuer Spalte
account_data3 = [
    (1,"otto","2019-10-01",4444,"neue Spalte 1")
]

schema = ["id","account","dt_transaction","balance"]
schema3 = ["id","account","dt_transaction","balance","new"]

df1 = spark.createDataFrame(data=account_data1, schema = schema).withColumn("dt_transaction",col("dt_transaction").cast("date")).repartition(3)
df2 = spark.createDataFrame(data=account_data2, schema = schema).withColumn("dt_transaction",col("dt_transaction").cast("date")).repartition(3)
df3 = spark.createDataFrame(data=account_data3, schema = schema3).withColumn("dt_transaction",col("dt_transaction").cast("date")).repartition(3)

print("++ create new dataframe and show schema and data")
print("################################################")

df1.printSchema()
df1.show(truncate=False)
df3.show(truncate=False)

++ create new dataframe and show schema and data
################################################
root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)

+---+-------+--------------+-------+
|id |account|dt_transaction|balance|
+---+-------+--------------+-------+
|2  |alex   |2019-02-01    |1500   |
|3  |alex   |2019-03-01    |1700   |
|1  |alex   |2019-01-01    |1000   |
|4  |maria  |2020-01-01    |5000   |
+---+-------+--------------+-------+

+---+-------+--------------+-------+-------------+
|id |account|dt_transaction|balance|new          |
+---+-------+--------------+-------+-------------+
|1  |otto   |2019-10-01    |4444   |neue Spalte 1|
+---+-------+--------------+-------+-------------+



## CSV

In [5]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_csv=(df1
           .write
           .format("csv")
           .mode("overwrite") # append
           .save("output/csv")
          )


Number of Partitions: 3


In [6]:
!ls output/csv/

_SUCCESS
part-00000-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv
part-00001-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv
part-00002-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv


In [9]:
! cat output/csv/part-00002-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv

4,maria,2020-01-01,5000


In [10]:
read_csv=spark.read.format("csv").load("output/csv")

read_csv.printSchema()
read_csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

+---+-----+----------+----+
|_c0|  _c1|       _c2| _c3|
+---+-----+----------+----+
|  2| alex|2019-02-01|1500|
|  3| alex|2019-03-01|1700|
|  4|maria|2020-01-01|5000|
|  1| alex|2019-01-01|1000|
+---+-----+----------+----+



In [11]:
write_csv=(df3
           .write
           .format("csv")
           .mode("append") # append
           .save("output/csv")
          )

In [12]:
!ls output/csv/

_SUCCESS
part-00000-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv
part-00000-411ddc45-144a-4a3d-900f-90a6bf3c46bf-c000.csv
part-00001-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv
part-00002-1969e3ad-db5e-4e0a-8214-4a9869c24d11-c000.csv
part-00002-411ddc45-144a-4a3d-900f-90a6bf3c46bf-c000.csv


In [16]:
!cat output/csv/part-00000-411ddc45-144a-4a3d-900f-90a6bf3c46bf-c000.csv

In [17]:
read_csv=spark.read.format("csv").load("output/csv")

read_csv.printSchema()
read_csv.show()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)

+---+-----+----------+----+
|_c0|  _c1|       _c2| _c3|
+---+-----+----------+----+
|  2| alex|2019-02-01|1500|
|  3| alex|2019-03-01|1700|
|  1| otto|2019-10-01|4444|
|  4|maria|2020-01-01|5000|
|  1| alex|2019-01-01|1000|
+---+-----+----------+----+



* kein Schema (Typen)
* kein anfügen neuer Spalten

## JSON

In [18]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_json=(df1
           .write
           .format("json")
           .mode("overwrite") # append
           .save("output/json")
          )


Number of Partitions: 3


In [19]:
!ls output/json/

_SUCCESS
part-00000-8f1197ca-4955-4b8a-a050-010b3eba7e2e-c000.json
part-00001-8f1197ca-4955-4b8a-a050-010b3eba7e2e-c000.json
part-00002-8f1197ca-4955-4b8a-a050-010b3eba7e2e-c000.json


In [20]:
! cat output/json/part-00000-8f1197ca-4955-4b8a-a050-010b3eba7e2e-c000.json

{"id":2,"account":"alex","dt_transaction":"2019-02-01","balance":1500}
{"id":3,"account":"alex","dt_transaction":"2019-03-01","balance":1700}


In [21]:
write_csv=(df3
           .write
           .format("json")
           .mode("append") # append
           .save("output/json")
          )

In [22]:
read_json=spark.read.format("json").load("output/json")

read_json.printSchema()
read_json.show()

root
 |-- account: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- dt_transaction: string (nullable = true)
 |-- id: long (nullable = true)
 |-- new: string (nullable = true)

+-------+-------+--------------+---+-------------+
|account|balance|dt_transaction| id|          new|
+-------+-------+--------------+---+-------------+
|   alex|   1500|    2019-02-01|  2|         null|
|   alex|   1700|    2019-03-01|  3|         null|
|   otto|   4444|    2019-10-01|  1|neue Spalte 1|
|  maria|   5000|    2020-01-01|  4|         null|
|   alex|   1000|    2019-01-01|  1|         null|
+-------+-------+--------------+---+-------------+



* Kein Schema
* Neue Spalten werden als neues Attribut hinzugefügt

## Avro

In [23]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_avro=(df1
           .write
           .format("avro")
           .mode("overwrite") # append
           .save("output/avro")
          )

Number of Partitions: 3


In [24]:
!ls output/avro/

_SUCCESS
part-00000-f47ea23a-bd11-4456-9c6e-dd455ffeecb3-c000.avro
part-00001-f47ea23a-bd11-4456-9c6e-dd455ffeecb3-c000.avro
part-00002-f47ea23a-bd11-4456-9c6e-dd455ffeecb3-c000.avro


In [25]:
! cat output/avro/part-00000-f47ea23a-bd11-4456-9c6e-dd455ffeecb3-c000.avro

Objavro.schema�{"type":"record","name":"topLevelRecord","fields":[{"name":"id","type":["long","null"]},{"name":"account","type":["string","null"]},{"name":"dt_transaction","type":[{"type":"int","logicalType":"date"},"null"]},{"name":"balance","type":["long","null"]}]}0org.apache.spark.version
3.1.2avro.codecsnappy ����!ɸ �9
Ht  alex �� �  alex Ș �3�������!ɸ �9


In [26]:
read_json=spark.read.format("avro").load("output/avro")
read_json.printSchema()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)



In [27]:
write_avro=(df3
           .write
           .format("avro")
           .mode("append") # append
           .save("output/avro")
          )

In [28]:
read_json=spark.read.format("avro").load("output/avro")
read_json.printSchema()
read_json.show()

root
 |-- id: long (nullable = true)
 |-- account: string (nullable = true)
 |-- dt_transaction: date (nullable = true)
 |-- balance: long (nullable = true)

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  1|   otto|    2019-10-01|   4444|
|  2|   alex|    2019-02-01|   1500|
|  3|   alex|    2019-03-01|   1700|
|  4|  maria|    2020-01-01|   5000|
|  1|   alex|    2019-01-01|   1000|
+---+-------+--------------+-------+



* Schema erhalten
* Schema evolution 

## Parquet

In [29]:
print("Number of Partitions:", df1.rdd.getNumPartitions())

write_parquet=(df1
           .write
           .partitionBy("account")
           .format("parquet")
           .mode("overwrite") # append
           .save("output/parquet")
          )


Number of Partitions: 3


In [24]:
df1.show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  2|   alex|    2019-02-01|   1500|
|  3|   alex|    2019-03-01|   1700|
|  1|   alex|    2019-01-01|   1000|
|  4|  maria|    2020-01-01|   5000|
+---+-------+--------------+-------+



In [33]:
!ls  output/parquet/
!echo "----------------------------"
!ls output/parquet/account=alex

_SUCCESS      account=alex  account=maria
----------------------------
part-00000-aacb8d79-4737-47c1-97a9-54c1dec538c3.c000.snappy.parquet
part-00001-aacb8d79-4737-47c1-97a9-54c1dec538c3.c000.snappy.parquet


In [34]:
! cat output/parquet/account=alex/part-00000-aacb8d79-4737-47c1-97a9-54c1dec538c3.c000.snappy.parquet

PAR1 ,.,                (                    ,             , $F  F   ($F  F     4   F  $F   ,0, �      �       (�      �         T   �      �      LHspark_schema %id %dt_transaction% %balance <&5 id��&<               (                    &�5 dt_transactionvz&�<$F  F   ($F  F        &�5 balance��&�<�      �       (�      �            � ,org.apache.spark.version3.1.2 )org.apache.spark.sql.parquet.row.metadata�{"type":"struct","fields":[{"name":"id","type":"long","nullable":true,"metadata":{}},{"name":"dt_transaction","type":"date","nullable":true,"metadata":{}},{"name":"balance","type":"long","nullable":true,"metadata":{}}]} Jparquet-mr version 1.10.1 (build a89df8f9932b6ef6633d06069e50c9b7970bebd1)<       �  PAR1

In [35]:
read_parquet=spark.read.format("parquet").load("output/parquet/").filter(col("account")=="alex").show()

+---+--------------+-------+-------+
| id|dt_transaction|balance|account|
+---+--------------+-------+-------+
|  2|    2019-02-01|   1500|   alex|
|  3|    2019-03-01|   1700|   alex|
|  1|    2019-01-01|   1000|   alex|
+---+--------------+-------+-------+



## Delta

In [57]:
write_delta=(df1
           .write
           .format("delta")
           .option("mergeSchema", "true")
           .mode("overwrite") # append
           .save("output/delta")
          )

In [39]:
! ls output/delta
! echo "-------------------------------------------------------------------"
! ls output/delta/_delta_log
! cat output/delta/_delta_log/00000000000000000003.json

_delta_log
part-00000-0c5a4784-da0d-4c7c-b061-b295aba553b9-c000.snappy.parquet
part-00000-79d8416d-1705-4cb8-ab50-a8d4262884db-c000.snappy.parquet
part-00000-ab2acb6a-065d-4f6e-bab6-af4d890690f5-c000.snappy.parquet
part-00000-e9b95674-8715-499f-bac3-89982b31cc30-c000.snappy.parquet
part-00001-005e4570-4e0b-4446-a401-5003d73a31d8-c000.snappy.parquet
part-00001-6352572a-10a1-4618-9ac6-eb76958b9ad4-c000.snappy.parquet
part-00002-13e80cc5-5ace-406d-a5b3-03a4092dd820-c000.snappy.parquet
part-00002-93171e79-a084-4195-ab1f-437fe91596e4-c000.snappy.parquet
part-00002-bb100df5-6963-44fa-a92c-8d91c90a2567-c000.snappy.parquet
part-00002-dba1b47f-e13c-41db-9135-12dfaea3ad84-c000.snappy.parquet
-------------------------------------------------------------------
00000000000000000000.json 00000000000000000002.json
00000000000000000001.json 00000000000000000003.json
{"commitInfo":{"timestamp":1681205640094,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"readVersion":

In [40]:
write_delta=(df2
           .write
           .format("delta")
           .mode("append") # append
           .save("output/delta")
          )



In [41]:
write_delta=(df3
           .write
           .format("delta")
           .option("mergeSchema", "true")
           .mode("overwrite") # append
           .save("output/delta")
          )

In [42]:
! ls output/delta
! echo "-------------------------------------------------------------------"
! ls output/delta/_delta_log
! cat output/delta/_delta_log/00000000000000000002.json

_delta_log
part-00000-0c5a4784-da0d-4c7c-b061-b295aba553b9-c000.snappy.parquet
part-00000-1d15fb39-e71c-405e-8895-62eeaf08245c-c000.snappy.parquet
part-00000-7438ebca-5702-42b3-b8d4-8749f5a0e63f-c000.snappy.parquet
part-00000-79d8416d-1705-4cb8-ab50-a8d4262884db-c000.snappy.parquet
part-00000-ab2acb6a-065d-4f6e-bab6-af4d890690f5-c000.snappy.parquet
part-00000-e9b95674-8715-499f-bac3-89982b31cc30-c000.snappy.parquet
part-00001-005e4570-4e0b-4446-a401-5003d73a31d8-c000.snappy.parquet
part-00001-6352572a-10a1-4618-9ac6-eb76958b9ad4-c000.snappy.parquet
part-00002-13e80cc5-5ace-406d-a5b3-03a4092dd820-c000.snappy.parquet
part-00002-265a7381-cab6-4143-8803-68852b3d1e52-c000.snappy.parquet
part-00002-6d9e00e3-7c08-4cdd-86a7-4070e4b03be3-c000.snappy.parquet
part-00002-93171e79-a084-4195-ab1f-437fe91596e4-c000.snappy.parquet
part-00002-bb100df5-6963-44fa-a92c-8d91c90a2567-c000.snappy.parquet
part-00002-dba1b47f-e13c-41db-9135-12dfaea3ad84-c000.snappy.parquet
-------------------------------------

In [43]:
deltaTable = DeltaTable.forPath(spark, "output/delta")

fullHistoryDF = deltaTable.history()    # get the full history of the table

In [44]:
fullHistoryDF.select("version","readVersion","timestamp","userId","operation","operationParameters","operationMetrics","userMetadata").show(truncate=True)

+-------+-----------+-------------------+------+---------+--------------------+--------------------+------------+
|version|readVersion|          timestamp|userId|operation| operationParameters|    operationMetrics|userMetadata|
+-------+-----------+-------------------+------+---------+--------------------+--------------------+------------+
|      5|          4|2023-04-11 11:39:56|  null|    WRITE|{mode -> Overwrit...|{numFiles -> 2, n...|        null|
|      4|          3|2023-04-11 11:39:39|  null|    WRITE|{mode -> Append, ...|{numFiles -> 2, n...|        null|
|      3|          2|2023-04-11 11:34:00|  null|    WRITE|{mode -> Overwrit...|{numFiles -> 3, n...|        null|
|      2|          1|2023-03-27 10:23:49|  null|    WRITE|{mode -> Overwrit...|{numFiles -> 2, n...|        null|
|      1|          0|2023-03-27 10:23:34|  null|    WRITE|{mode -> Append, ...|{numFiles -> 2, n...|        null|
|      0|       null|2023-03-27 10:22:22|  null|    WRITE|{mode -> Overwrit...|{numFiles

In [50]:
read_parquet=spark.read.format("delta").load("output/delta/").show()

+---+-------+--------------+-------+-------------+
| id|account|dt_transaction|balance|          new|
+---+-------+--------------+-------+-------------+
|  1|   otto|    2019-10-01|   4444|neue Spalte 1|
+---+-------+--------------+-------+-------------+



## Time travel

In [48]:
spark.read.format("delta").option("versionAsOf", "4").load("output/delta").show()


+---+-------+--------------+-------+----+
| id|account|dt_transaction|balance| new|
+---+-------+--------------+-------+----+
|  2|   alex|    2019-02-01|   1500|null|
|  3|   alex|    2019-03-01|   1700|null|
|  2|  peter|    2021-01-01|    100|null|
|  4|  maria|    2020-01-01|   5000|null|
|  1|   alex|    2019-03-01|   3300|null|
|  1|   alex|    2019-01-01|   1000|null|
+---+-------+--------------+-------+----+



## Merge

In [58]:
deltaTable2 = DeltaTable.forPath(spark, "output/delta")

df2.show()
deltaTable2.toDF().show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  1|   alex|    2019-03-01|   3300|
|  2|  peter|    2021-01-01|    100|
+---+-------+--------------+-------+

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  2|   alex|    2019-02-01|   1500|
|  3|   alex|    2019-03-01|   1700|
|  4|  maria|    2020-01-01|   5000|
|  1|   alex|    2019-01-01|   1000|
+---+-------+--------------+-------+



In [59]:
dt3=(deltaTable2.alias("oldData")
      .merge(df2.alias("newData"),
            "oldData.account = newData.account AND oldData.dt_transaction = newData.dt_transaction")
            .whenMatchedUpdateAll()
            .whenNotMatchedInsertAll()
      .execute()
    )

deltaTable2.toDF().show()

+---+-------+--------------+-------+
| id|account|dt_transaction|balance|
+---+-------+--------------+-------+
|  2|  peter|    2021-01-01|    100|
|  4|  maria|    2020-01-01|   5000|
|  1|   alex|    2019-01-01|   1000|
|  2|   alex|    2019-02-01|   1500|
|  1|   alex|    2019-03-01|   3300|
+---+-------+--------------+-------+



In [60]:
result=(deltaTable2
        .toDF()
        .withColumn("month",month(col("dt_transaction")))
        .groupBy("account","month").agg(sum("balance"))
        .sort("account","month")
       )
result.show()

+-------+-----+------------+
|account|month|sum(balance)|
+-------+-----+------------+
|   alex|    1|        1000|
|   alex|    2|        1500|
|   alex|    3|        3300|
|  maria|    1|        5000|
|  peter|    1|         100|
+-------+-----+------------+



In [75]:
result=(spark.read
        .format("delta")
        .option("versionAsOf", "1")
        .load("output/delta")
        .withColumn("month",month(col("dt_transaction")))
        .groupBy("account","month").agg(sum("balance"))
        .sort("account","month")
       )
result.show()

+-------+-----+------------+
|account|month|sum(balance)|
+-------+-----+------------+
|   alex|    1|        1000|
|   alex|    2|        1500|
|   alex|    3|        3300|
|  maria|    1|        5000|
|  peter|    1|         100|
+-------+-----+------------+



* Schema
* Schema evolution
* Transaction Log
* Time Travel

In [48]:
spark.stop()